In [1]:
import numpy as np
import pandas as pd

In [55]:
df = pd.read_csv('IMDB_Dataset.csv')
num_rows_org = df.shape[0]
print(num_rows_org)

50000


In [57]:
pattern = '[^\x00-\x7F]+'

filtered_df = df[~df['title'].str.contains(pattern)]

# Save the filtered DataFrame back to a CSV file
filtered_df.to_csv('IMDB_Dataset.csv2', index=False)

In [59]:
df = pd.read_csv('IMDB_Dataset.csv')

In [61]:
senti_counts = df['sentiment'].value_counts()
print(senti_counts)

sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [63]:
df.isnull().sum()

title        0
sentiment    0
dtype: int64

In [65]:
'''import spacy

nlp = spacy.load('en_core_web_sm')

def tokenizer(text):
  token = [token.lemma_ for token in nlp(text)]
  return ' '.join(token)'''

"import spacy\n\nnlp = spacy.load('en_core_web_sm')\n\ndef tokenizer(text):\n  token = [token.lemma_ for token in nlp(text)]\n  return ' '.join(token)"

In [67]:
"""df['preprocessed_text'] = df['review'].apply(tokenizer)

# Preview the preprocessed data
print(df[['Title', 'preprocessed_text']].head())"""

"df['preprocessed_text'] = df['review'].apply(tokenizer)\n\n# Preview the preprocessed data\nprint(df[['Title', 'preprocessed_text']].head())"

In [69]:
# selecting features
X = df[['title']]
y = df[['sentiment']]

In [71]:
from sklearn.model_selection import train_test_split

# splitting our data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 45)

# checking our train, test data to see for any inconsistencies
print('X - Training Data Shape: ', X_train.shape)
print('y - Training Data Shape: ', y_train.shape)
print('X - Testing Data Shape: ', X_test.shape)
print('y - Testing Data Shape: ', y_test.shape)

X - Training Data Shape:  (33500, 1)
y - Training Data Shape:  (33500, 1)
X - Testing Data Shape:  (16500, 1)
y - Testing Data Shape:  (16500, 1)


In [73]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

# Naive Bayes
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

# Linear SVC
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

# Logistic Regression
text_clf_lr_lbfgs = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression(solver='lbfgs', max_iter=1000))])
# setting max_iter = 1000 - since it wasn't converging with the default no. of iterations

In [75]:
text_clf_nb.fit(X_train['title'], y_train['sentiment'])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [77]:
predictions_nb = text_clf_nb.predict(X_test['title'])

In [79]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test['sentiment'], predictions_nb))

[[7243  987]
 [1359 6911]]


In [81]:
print(metrics.accuracy_score(y_test['sentiment'], predictions_nb))
print('\n')

0.8578181818181818




In [83]:
text_clf_lsvc.fit(X_train['title'], y_train['sentiment'])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [85]:
predictions_lsvc = text_clf_lsvc.predict(X_test['title'])

In [87]:
print(metrics.confusion_matrix(y_test['sentiment'], predictions_lsvc))

[[7319  911]
 [ 798 7472]]


In [89]:
print(metrics.accuracy_score(y_test['sentiment'], predictions_lsvc))

0.8964242424242425


In [101]:
# Function to predict sentiment using the pipeline
def predict_sentiment_svc(user_input):
    # Predict the sentiment directly using the pipeline
    predicted_sentiment = text_clf_lsvc.predict([user_input])[0]

    return predicted_sentiment

# Function to generate rule-based responses based on predicted sentiment
def generate_rule_based_response(predicted_sentiment):
    if predicted_sentiment == "positive":
        response = "That's positive news! I'm happy for you!"
    elif predicted_sentiment == "negative":
        response = "I'm sorry for the negative news!"
    return response

# Function to handle the chatbot interaction by integrating sentiment prediction
def generate_rule_based_response_chatbot(user_input):
    # Predict sentiment using the pipeline
    predicted_sentiment = predict_sentiment_svc(user_input)

    # Generate response based on predicted sentiment using rule-based approach
    response = generate_rule_based_response(predicted_sentiment)

    return response

# Function to handle pattern-based responses
def generate_pattern_response(user_input):
    patterns = {
        "hello": "Hello! How can I assist you today?",
        "how are you": "I'm just a chatbot, but I'm here to help! How can I assist you?",
        "help": "Sure, I'd be happy to help. What do you need assistance with?",
        "bye": "Goodbye! If you have more questions in the future, feel free to ask.",
    }

    # Look for pattern matches and return the corresponding response
    for pattern, response in patterns.items():
        if pattern in user_input.lower():
            return response

    # If no pattern matches, use the rule-based response based on sentiment
    return generate_rule_based_response_chatbot(user_input)


In [107]:
#loop to interact with the user
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye!")
        break
    bot_response = generate_pattern_response(user_input)
    print("Bot:", bot_response)

You:  hello


Bot: Hello! How can I assist you today?


You:  i started my day quite well


Bot: That's positive news! I'm happy for you!


You:  i have a lot of work to do


...

Bot: I'm sorry for the negative news!


You:  have a great day


Bot: That's positive news! I'm happy for you!


You:  bye


Bot: Goodbye!
